In [1]:
import pandas as pd
import pymongo
from pymongo.errors import PyMongoError
import os

# Get the MongoDB connection details from environment variables
mongo_host = os.environ.get('MONGO_HOST', 'localhost')
mongo_port = int(os.environ.get('MONGO_PORT', 27017))
mongo_username = os.environ.get('MONGO_USERNAME', 'root')
mongo_password = os.environ.get('MONGO_PASSWORD', 'example')
mongo_auth_source = os.environ.get('MONGO_AUTH_SOURCE', 'admin')

# MongoDB connection string
mongo_uri = f'mongodb://{mongo_username}:{mongo_password}@{mongo_host}:{mongo_port}/{mongo_auth_source}'

# Connect to MongoDB
client = pymongo.MongoClient(mongo_uri)

# Create or access the pp_db database and pp_data_col collection
pp_db = client.pp_db
pp_data_col = pp_db.pp_data

# Clear pp_data_col collectoin
pp_data_col.delete_many({})

# READ DATA FROM CSV FILE
# For server
pp_df = pd.read_csv("/usr/src/data/global_power_plant_database.csv", low_memory=False)

# For local (if you are using a jupyter server in your local)
# pp_df = pd.read_csv("../../data/global_power_plant_database.csv", low_memory=False)

transferred_documents = 0

# Extract each row from the dataframe, do cleaning and transformations and load to MongoDB
for index, row in pp_df.iterrows():
    template = {
        'country_id': '',
        'country_name': '',
        'plant_name': '',
        'gppd_id': '',
        'capacity_mw': 0.0,
        'latitude': 0.0,
        'longitude': 0.0,
        'fuel_source': {
            'primary': '',
            'secondary': '',
            'tertiary': '',
            'quaternary': ''
        },
        'commissioning_year': '',
        'owner': '',
        'data_source': '',
        'data_source_url': '',
        'actual_generation': {
            'generation_gwh_2013': 0.0,
            'generation_gwh_2014': 0.0,
            'generation_gwh_2015': 0.0,
            'generation_gwh_2016': 0.0,
            'generation_gwh_2017': 0.0,
            'generation_gwh_2018': 0.0,
            'generation_gwh_2019': 0.0
        },
        'estimated_generation': {
            'estimated_generation_gwh_2013': 0.0,
            'estimated_generation_gwh_2014': 0.0,
            'estimated_generation_gwh_2015': 0.0,
            'estimated_generation_gwh_2016': 0.0,
            'estimated_generation_gwh_2017': 0.0
        }
    }

    # Transferring values
    template['country_id'] = row['country'] if not pd.isna(row['country']) else None
    template['country_name'] = row['country_long']
    template['plant_name'] = row['name']
    template['gppd_id'] = row['gppd_idnr']
    template['capacity_mw'] = float(row['capacity_mw'])
    template['latitude'] = float(row['latitude'])
    template['longitude'] = float(row['longitude'])
    template['fuel_source']['primary'] = row['primary_fuel']
    template['fuel_source']['secondary'] = row['other_fuel1'] if not pd.isna(row['other_fuel1']) else None
    template['fuel_source']['tertiary'] = row['other_fuel2'] if not pd.isna(row['other_fuel2']) else None
    template['fuel_source']['quatenary'] = row['other_fuel3'] if not pd.isna(row['other_fuel3']) else None
    template['commissioning_year'] = row['commissioning_year'] if not pd.isna(row['commissioning_year']) else None
    template['owner'] = row['owner'] if not pd.isna(row['owner']) else None
    template['data_source'] = row['source'] if not pd.isna(row['source']) else None
    template['data_source_url'] = row['url'] if not pd.isna(row['url']) else None
    template['actual_generation']['generation_gwh_2013'] = float(row['generation_gwh_2013']) if not pd.isna(row['generation_gwh_2013']) else None
    template['actual_generation']['generation_gwh_2014'] = float(row['generation_gwh_2014']) if not pd.isna(row['generation_gwh_2014']) else None
    template['actual_generation']['generation_gwh_2015'] = float(row['generation_gwh_2015']) if not pd.isna(row['generation_gwh_2015']) else None
    template['actual_generation']['generation_gwh_2016'] = float(row['generation_gwh_2016']) if not pd.isna(row['generation_gwh_2016']) else None
    template['actual_generation']['generation_gwh_2017'] = float(row['generation_gwh_2017']) if not pd.isna(row['generation_gwh_2017']) else None
    template['actual_generation']['generation_gwh_2018'] = float(row['generation_gwh_2018']) if not pd.isna(row['generation_gwh_2018']) else None
    template['actual_generation']['generation_gwh_2019'] = float(row['generation_gwh_2019']) if not pd.isna(row['generation_gwh_2019']) else None
    template['estimated_generation']['estimated_generation_gwh_2013'] = float(row['estimated_generation_gwh_2013']) if not pd.isna(row['estimated_generation_gwh_2013']) else None
    template['estimated_generation']['estimated_generation_gwh_2014'] = float(row['estimated_generation_gwh_2014']) if not pd.isna(row['estimated_generation_gwh_2014']) else None
    template['estimated_generation']['estimated_generation_gwh_2015'] = float(row['estimated_generation_gwh_2015']) if not pd.isna(row['estimated_generation_gwh_2015']) else None
    template['estimated_generation']['estimated_generation_gwh_2016'] = float(row['estimated_generation_gwh_2016']) if not pd.isna(row['estimated_generation_gwh_2016']) else None
    template['estimated_generation']['estimated_generation_gwh_2017'] = float(row['estimated_generation_gwh_2017']) if not pd.isna(row['estimated_generation_gwh_2017']) else None

    try:
        result = pp_data_col.insert_one(template)

        # Print the inserted document's ID with row ID
        print(f"Document " + str(index) + " inserted with ID: " + str(result.inserted_id))
        transferred_documents += 1
    

    except PyMongoError as e:
        print(f"An error occurred: {e}")

print(str(transferred_documents) + " out of " + str(len(pp_df)))

Document 0 inserted with ID: 65a7ccd716f98754296a1e07
Document 1 inserted with ID: 65a7ccd716f98754296a1e08
Document 2 inserted with ID: 65a7ccd716f98754296a1e09
Document 3 inserted with ID: 65a7ccd716f98754296a1e0a
Document 4 inserted with ID: 65a7ccd716f98754296a1e0b
Document 5 inserted with ID: 65a7ccd716f98754296a1e0c
Document 6 inserted with ID: 65a7ccd716f98754296a1e0d
Document 7 inserted with ID: 65a7ccd716f98754296a1e0e
Document 8 inserted with ID: 65a7ccd716f98754296a1e0f
Document 9 inserted with ID: 65a7ccd716f98754296a1e10
Document 10 inserted with ID: 65a7ccd716f98754296a1e11
Document 11 inserted with ID: 65a7ccd716f98754296a1e12
Document 12 inserted with ID: 65a7ccd716f98754296a1e13
Document 13 inserted with ID: 65a7ccd716f98754296a1e14
Document 14 inserted with ID: 65a7ccd716f98754296a1e15
Document 15 inserted with ID: 65a7ccd716f98754296a1e16
Document 16 inserted with ID: 65a7ccd716f98754296a1e17
Document 17 inserted with ID: 65a7ccd716f98754296a1e18
Document 18 inserted